In [1]:
import pandas as pd
import pickle
from shapely.geometry import LineString

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
plays = pd.read_csv("../processed_data/clean/plays.csv",index_col=0)
plays.head()

,id,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018090600:75:1,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018090600:146:1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False
2,2018090600:168:1,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False
3,2018090600:190:1,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False
4,2018090600:256:1,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",0.0,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False


In [4]:
games = pd.read_csv("../raw_data/games.csv",index_col=0)
games.head()

,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week
gameId,,,,,
2018090600,09/06/2018,20:20:00,PHI,ATL,1
2018090901,09/09/2018,13:00:00,CLE,PIT,1
2018090902,09/09/2018,13:00:00,IND,CIN,1
2018090903,09/09/2018,13:00:00,MIA,TEN,1
2018090900,09/09/2018,13:00:00,BAL,BUF,1


In [5]:
week1 = pd.read_csv("../processed_data/clean/week1.csv",index_col=5)
week1

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,level_0,Unnamed: 0.1.1.1,id,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,2018090600:75:1,2018-09-07T01:07:14.599Z,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,Matt Ryan,2.0,QB,1,away,left,NaN
1,1,1,1,1,1,2018090600:75:1,2018-09-07T01:07:14.599Z,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,Malcolm Jenkins,27.0,SS,1,home,left,NaN
2,2,2,2,2,2,2018090600:75:1,2018-09-07T01:07:14.599Z,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,left,HITCH
3,3,3,3,3,3,2018090600:75:1,2018-09-07T01:07:14.599Z,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,Corey Graham,24.0,FS,1,home,left,NaN
4,4,4,4,4,4,2018090600:75:1,2018-09-07T01:07:14.599Z,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,left,HITCH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986017,983363,985510,985510,986017,986017,2018091001:4192:1,2018-09-11T05:29:39.900Z,33.68,43.14,1.96,3.79,0.22,281.35,309.70,None,2556277.0,Marqui Christian,41.0,SS,65,away,left,NaN
986018,983364,985511,985511,986018,986018,2018091001:4192:1,2018-09-11T05:29:39.900Z,36.02,40.21,0.81,1.52,0.09,308.07,234.06,None,2556593.0,Cory Littleton,58.0,ILB,65,away,left,NaN
986019,983365,985512,985512,986019,986019,2018091001:4192:1,2018-09-11T05:29:39.900Z,33.02,43.81,2.63,4.57,0.27,183.83,314.39,None,2556885.0,Jalen Richard,30.0,RB,65,home,left,SCREEN


In [6]:
week1.drop(["Unnamed: 0","Unnamed: 0.1","Unnamed: 0.1.1","Unnamed: 0.1.1.1","level_0"],axis=1,inplace=True)

In [7]:
week1.drop(["time"],axis=1,inplace=True)

In [8]:
week1.head()

,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
0,2018090600:75:1,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,Matt Ryan,2.0,QB,1,away,left,NaN
1,2018090600:75:1,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,Malcolm Jenkins,27.0,SS,1,home,left,NaN
2,2018090600:75:1,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,left,HITCH
3,2018090600:75:1,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,Corey Graham,24.0,FS,1,home,left,NaN
4,2018090600:75:1,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,left,HITCH


In [9]:
#save dataset
pickle_file = open('../resources/all_plays.pickle','wb')
pickle.dump(plays["id"].values,pickle_file)
pickle_file.close()

In [10]:
def obtainDefenseTeam(playId):
    playIdParts = playId.split(":")
    gameId = playIdParts[0]
    visitorTeam = games.loc[int(gameId)]["visitorTeamAbbr"]
    homeTeam = games.loc[int(gameId)]["homeTeamAbbr"]
    possessionTeam = plays[plays["id"]==playId]["possessionTeam"].values[0]
    
    if possessionTeam == visitorTeam: #si equipo visitante tiene el balón, el equipo local es el que defiende
        value = "home"
    elif possessionTeam == homeTeam:
        value = "away"
    
    return value

In [11]:
def obtainOffenseTeam(playId):
    playIdParts = playId.split(":")
    gameId = playIdParts[0]
    visitorTeam = games.loc[int(gameId)]["visitorTeamAbbr"]
    homeTeam = games.loc[int(gameId)]["homeTeamAbbr"]
    possessionTeam = plays[plays["id"]==playId]["possessionTeam"].values[0]
    
    if possessionTeam == visitorTeam: #si equipo visitante tiene el balón, el equipo local es el que defiende
        value = "away"
    elif possessionTeam == homeTeam:
        value = "home"
    
    return value

In [12]:
obtainDefenseTeam("2018090600:1037:1")

'away'

Hay que secuenciar al menos un jugador

In [13]:
selected_play = week1[week1["id"]=="2018090600:1037:1"]
selected_play[selected_play["frameId"]==1]

,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
15508,2018090600:1037:1,68.36,35.20,0.00,0.00,0.00,257.18,129.73,None,2507763.0,Mike Wallace,14.0,WR,1,home,left,HITCH
15509,2018090600:1037:1,72.08,23.78,0.00,0.00,0.00,265.74,275.93,None,2532842.0,Nick Foles,9.0,QB,1,home,left,NaN
15510,2018090600:1037:1,66.63,35.21,0.00,0.00,0.00,101.79,44.87,None,2539334.0,Desmond Trufant,21.0,CB,1,away,left,NaN
15511,2018090600:1037:1,65.58,15.28,0.01,0.01,0.01,75.79,295.44,None,2539653.0,Robert Alford,23.0,CB,1,away,left,NaN
15512,2018090600:1037:1,69.61,17.26,0.00,0.01,0.00,188.41,7.62,None,2540158.0,Zach Ertz,86.0,TE,1,home,left,OUT
15513,2018090600:1037:1,51.97,27.53,0.41,0.31,0.04,95.89,286.70,None,2543850.0,Ricardo Allen,37.0,SS,1,away,left,NaN
15514,2018090600:1037:1,72.02,21.53,0.00,0.00,0.00,267.57,135.02,None,2552582.0,Jay Ajayi,26.0,RB,1,home,left,NaN
15515,2018090600:1037:1,68.66,15.83,0.00,0.00,0.00,250.65,266.80,None,2552600.0,Nelson Agholor,13.0,WR,1,home,left,HITCH
15516,2018090600:1037:1,68.82,29.58,0.00,0.00,0.00,268.50,151.21,None,2553502.0,DeAndre Carter,16.0,WR,1,home,left,FLAT


In [14]:
selected_play[(selected_play["team"]==obtainDefenseTeam("2018090600:1037:1")) & (selected_play["frameId"]==1)]

,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
15510,2018090600:1037:1,66.63,35.21,0.00,0.00,0.00,101.79,44.87,None,2539334.0,Desmond Trufant,21.0,CB,1,away,left,NaN
15511,2018090600:1037:1,65.58,15.28,0.01,0.01,0.01,75.79,295.44,None,2539653.0,Robert Alford,23.0,CB,1,away,left,NaN
15513,2018090600:1037:1,51.97,27.53,0.41,0.31,0.04,95.89,286.70,None,2543850.0,Ricardo Allen,37.0,SS,1,away,left,NaN
15517,2018090600:1037:1,65.75,21.74,0.00,0.00,0.00,92.67,103.82,None,2555162.0,Deion Jones,45.0,MLB,1,away,left,NaN
15518,2018090600:1037:1,66.90,17.19,0.00,0.01,0.00,95.69,338.77,None,2555255.0,De'Vondre Campbell,59.0,OLB,1,away,left,NaN
15519,2018090600:1037:1,57.77,22.98,0.27,1.17,0.03,126.24,210.03,None,2555543.0,Keanu Neal,22.0,FS,1,away,left,NaN
15520,2018090600:1037:1,65.43,28.96,0.93,0.62,0.08,96.78,190.65,None,2556445.0,Brian Poole,34.0,CB,1,away,left,NaN


In [15]:
unique_player = selected_play[selected_play["nflId"]==2539334]
unique_player

,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
15510,2018090600:1037:1,66.63,35.21,0.00,0.00,0.00,101.79,44.87,None,2539334.0,Desmond Trufant,21.0,CB,1,away,left,NaN
15524,2018090600:1037:1,66.63,35.21,0.00,0.00,0.00,101.79,54.12,None,2539334.0,Desmond Trufant,21.0,CB,2,away,left,NaN
15538,2018090600:1037:1,66.62,35.21,0.00,0.00,0.00,101.79,60.58,None,2539334.0,Desmond Trufant,21.0,CB,3,away,left,NaN
15552,2018090600:1037:1,66.62,35.21,0.00,0.00,0.00,101.79,165.93,None,2539334.0,Desmond Trufant,21.0,CB,4,away,left,NaN
15566,2018090600:1037:1,66.62,35.21,0.00,0.00,0.00,101.79,177.13,None,2539334.0,Desmond Trufant,21.0,CB,5,away,left,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16336,2018090600:1037:1,51.68,42.84,2.10,1.78,0.21,79.95,83.24,pass_outcome_caught,2539334.0,Desmond Trufant,21.0,CB,60,away,left,NaN
16350,2018090600:1037:1,51.89,42.86,2.08,1.47,0.21,83.15,87.78,None,2539334.0,Desmond Trufant,21.0,CB,61,away,left,NaN
16364,2018090600:1037:1,52.09,42.86,2.04,1.39,0.21,85.51,92.31,None,2539334.0,Desmond Trufant,21.0,CB,62,away,left,NaN


In [16]:
selected_play[selected_play["nflId"]==2539334]["event"].unique()

array(['None', 'ball_snap', 'pass_forward', 'pass_arrived',
       'pass_outcome_caught', 'tackle'], dtype=object)

In [17]:
selected_play[(selected_play["nflId"]==2539334) & (selected_play["event"]=="ball_snap")]["frameId"].values[0]

11

In [18]:
selected_play[selected_play["event"]=="pass_forward"]

,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
16152,2018090600:1037:1,52.49,41.96,3.89,6.12,0.43,200.40,280.43,pass_forward,2507763.0,Mike Wallace,14.0,WR,47,home,left,HITCH
16153,2018090600:1037:1,76.17,22.30,1.78,1.33,0.19,241.92,219.83,pass_forward,2532842.0,Nick Foles,9.0,QB,47,home,left,NaN
16154,2018090600:1037:1,52.54,41.42,5.28,4.91,0.55,296.24,283.67,pass_forward,2539334.0,Desmond Trufant,21.0,CB,47,away,left,NaN
16155,2018090600:1037:1,54.59,16.21,0.62,6.68,0.08,359.87,294.71,pass_forward,2539653.0,Robert Alford,23.0,CB,47,away,left,NaN
16156,2018090600:1037:1,54.29,24.45,4.69,5.14,0.48,196.63,259.43,pass_forward,2540158.0,Zach Ertz,86.0,TE,47,home,left,OUT
16157,2018090600:1037:1,43.94,33.11,4.54,1.01,0.46,57.92,316.20,pass_forward,2543850.0,Ricardo Allen,37.0,SS,47,away,left,NaN
16158,2018090600:1037:1,75.34,23.80,2.08,5.02,0.21,270.50,121.03,pass_forward,2552582.0,Jay Ajayi,26.0,RB,47,home,left,NaN
16159,2018090600:1037:1,55.51,16.49,0.61,5.10,0.06,47.53,174.69,pass_forward,2552600.0,Nelson Agholor,13.0,WR,47,home,left,HITCH
16160,2018090600:1037:1,68.32,41.26,5.70,1.94,0.58,89.15,357.36,pass_forward,2553502.0,DeAndre Carter,16.0,WR,47,home,left,FLAT


In [19]:
selected_play.drop(selected_play[(selected_play["frameId"]<11)|(selected_play["frameId"]>47)].index,inplace=True)

/home/villamor_coto/.local/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [20]:
selected_play

,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
15648,2018090600:1037:1,68.36,35.20,0.00,0.00,0.00,257.83,265.68,ball_snap,2507763.0,Mike Wallace,14.0,WR,11,home,left,HITCH
15649,2018090600:1037:1,72.08,23.78,0.00,0.00,0.00,265.74,275.50,ball_snap,2532842.0,Nick Foles,9.0,QB,11,home,left,NaN
15650,2018090600:1037:1,66.62,35.20,0.00,0.00,0.00,101.79,197.17,ball_snap,2539334.0,Desmond Trufant,21.0,CB,11,away,left,NaN
15651,2018090600:1037:1,65.59,15.30,0.00,0.00,0.01,73.64,308.71,ball_snap,2539653.0,Robert Alford,23.0,CB,11,away,left,NaN
15652,2018090600:1037:1,69.45,17.12,0.23,0.09,0.21,309.85,231.38,ball_snap,2540158.0,Zach Ertz,86.0,TE,11,home,left,OUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16161,2018090600:1037:1,74.19,23.09,3.62,3.72,0.34,117.95,152.92,pass_forward,2555162.0,Deion Jones,45.0,MLB,47,away,left,NaN
16162,2018090600:1037:1,54.85,24.76,5.19,4.74,0.54,240.88,274.87,pass_forward,2555255.0,De'Vondre Campbell,59.0,OLB,47,away,left,NaN
16163,2018090600:1037:1,44.82,21.17,3.40,3.72,0.36,95.92,260.10,pass_forward,2555543.0,Keanu Neal,22.0,FS,47,away,left,NaN


In [21]:
unique_player.drop(unique_player[(unique_player["frameId"]<11)|(unique_player["frameId"]>47)].index,inplace=True)
unique_player

/home/villamor_coto/.local/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
15650,2018090600:1037:1,66.62,35.20,0.00,0.00,0.00,101.79,197.17,ball_snap,2539334.0,Desmond Trufant,21.0,CB,11,away,left,NaN
15664,2018090600:1037:1,66.62,35.20,0.00,0.00,0.00,101.79,197.33,None,2539334.0,Desmond Trufant,21.0,CB,12,away,left,NaN
15678,2018090600:1037:1,66.62,35.20,0.01,0.22,0.00,101.79,199.46,None,2539334.0,Desmond Trufant,21.0,CB,13,away,left,NaN
15692,2018090600:1037:1,66.62,35.20,0.04,0.33,0.00,102.44,207.25,None,2539334.0,Desmond Trufant,21.0,CB,14,away,left,NaN
15706,2018090600:1037:1,66.61,35.19,0.07,0.34,0.01,101.69,241.21,None,2539334.0,Desmond Trufant,21.0,CB,15,away,left,NaN
15720,2018090600:1037:1,66.60,35.19,0.15,0.53,0.02,101.10,246.38,None,2539334.0,Desmond Trufant,21.0,CB,16,away,left,NaN
15734,2018090600:1037:1,66.57,35.20,0.25,0.74,0.03,99.98,276.57,None,2539334.0,Desmond Trufant,21.0,CB,17,away,left,NaN
15748,2018090600:1037:1,66.53,35.22,0.45,1.42,0.05,94.33,295.16,None,2539334.0,Desmond Trufant,21.0,CB,18,away,left,NaN
15762,2018090600:1037:1,66.48,35.26,0.66,1.64,0.06,91.28,300.35,None,2539334.0,Desmond Trufant,21.0,CB,19,away,left,NaN


In [22]:
selected_play["frameId"].unique()

array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
       28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44,
       45, 46, 47])

In [23]:
selected_play[(selected_play["frameId"]==11)&(selected_play["team"]==obtainOffenseTeam("2018090600:1037:1"))&(selected_play["route"].notnull())]

,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
15648,2018090600:1037:1,68.36,35.20,0.00,0.00,0.00,257.83,265.68,ball_snap,2507763.0,Mike Wallace,14.0,WR,11,home,left,HITCH
15652,2018090600:1037:1,69.45,17.12,0.23,0.09,0.21,309.85,231.38,ball_snap,2540158.0,Zach Ertz,86.0,TE,11,home,left,OUT
15655,2018090600:1037:1,68.65,15.83,0.00,0.00,0.00,247.46,257.09,ball_snap,2552600.0,Nelson Agholor,13.0,WR,11,home,left,HITCH
15656,2018090600:1037:1,68.82,29.58,0.00,0.00,0.00,269.12,200.68,ball_snap,2553502.0,DeAndre Carter,16.0,WR,11,home,left,FLAT


In [24]:
for frame in selected_play["frameId"].unique():
    players = selected_play[(selected_play["frameId"]==frame)&(selected_play["team"]==obtainOffenseTeam("2018090600:1037:1"))&(selected_play["route"].notnull())]
    x = unique_player[unique_player["frameId"]==frame]["x"].values[0]
    y = unique_player[unique_player["frameId"]==frame]["y"].values[0]
    distances = []
    for index,row in players.iterrows():
        x_value = row["x"]
        y_value = row["y"]
        line = LineString([(x, y), (x_value, y_value)]) #distancia euclidea
        distances.append(line.length) #habría que tener en cuenta el jugador???
    print(min(distances))

1.7399999999999949
1.7299999999999898
1.7300289014926789
1.7200290695217906
1.6900295855398486
1.6301226947687193
1.5608010763707227
1.4558159224297584
1.3520355024924349
1.2614673994994918
1.2016655108639853
1.1359577456930305
1.132254388377445
1.1669190203266075
1.229186723000215
1.2870120434556938
1.3776792079435674
1.446996890114144
1.4960280746028767
1.5220052562327115
1.5158495967608465
1.488522757635906
1.4439182802361072
1.3652838532700802
1.2701574705523753
1.172731853408956
1.0850345616615171
1.01921538449927
0.9638464608017188
0.9213034245024827
0.8781799360040092
0.8490583018850911
0.8163332652783393
0.7647875521999561
0.7038465741907108
0.6140032573203538
0.5423098745182489


Podemos sacar la distancia de marcaje de un determinado jugador

Haremos lo mismo pero ahora con el safety

In [31]:
unique_player_safety = selected_play[selected_play["nflId"]==2543850]
unique_player_safety

,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
15653,2018090600:1037:1,51.59,27.65,0.42,0.06,0.04,95.60,283.14,ball_snap,2543850.0,Ricardo Allen,37.0,SS,11,away,left,NaN
15667,2018090600:1037:1,51.55,27.65,0.42,0.32,0.04,97.59,272.17,None,2543850.0,Ricardo Allen,37.0,SS,12,away,left,NaN
15681,2018090600:1037:1,51.50,27.65,0.49,0.58,0.05,98.83,262.90,None,2543850.0,Ricardo Allen,37.0,SS,13,away,left,NaN
15695,2018090600:1037:1,51.45,27.64,0.61,0.92,0.05,101.01,255.07,None,2543850.0,Ricardo Allen,37.0,SS,14,away,left,NaN
15709,2018090600:1037:1,51.38,27.62,0.81,1.30,0.07,103.10,247.66,None,2543850.0,Ricardo Allen,37.0,SS,15,away,left,NaN
15723,2018090600:1037:1,51.29,27.58,1.07,1.60,0.10,107.61,248.21,None,2543850.0,Ricardo Allen,37.0,SS,16,away,left,NaN
15737,2018090600:1037:1,51.18,27.54,1.32,1.75,0.12,109.96,249.49,None,2543850.0,Ricardo Allen,37.0,SS,17,away,left,NaN
15751,2018090600:1037:1,51.04,27.49,1.56,1.78,0.15,106.07,252.81,None,2543850.0,Ricardo Allen,37.0,SS,18,away,left,NaN
15765,2018090600:1037:1,50.87,27.45,1.85,1.88,0.18,107.42,254.50,None,2543850.0,Ricardo Allen,37.0,SS,19,away,left,NaN


In [26]:
for frame in selected_play["frameId"].unique():
    players = selected_play[(selected_play["frameId"]==frame)&(selected_play["team"]==obtainOffenseTeam("2018090600:1037:1"))&(selected_play["route"].notnull())]
    x = unique_player[unique_player["frameId"]==frame]["x"].values[0]
    y = unique_player[unique_player["frameId"]==frame]["y"].values[0]
    distances = []
    for index,row in players.iterrows():
        x_value = row["x"]
        y_value = row["y"]
        line = LineString([(x, y), (x_value, y_value)]) #distancia euclidea
        distances.append(line.length) #habría que tener en cuenta el jugador???
    print(min(distances))

17.33775648692759
17.367570353967196
17.41615629236256
17.46585239832285
17.534337740559238
17.6138638577684
17.72995487867919
17.894848420704776
18.100723742436376
18.34558530001156
18.62335093370685
18.933855920017976
19.19296746206797
19.02899103998949
18.80724594405039
18.59081762591414
18.37881933095812
18.16894328242565
17.942577852694406
17.706072404686473
17.42436512473267
17.121962504339272
16.791119676781538
16.445205988372418
16.072551135398513
15.680624987544345
15.272891016438248
14.882183979510538
14.485254571459905
14.141177461583604
13.84234445460739
13.59479679877563
13.422257634243207
13.285796927546349
12.910557695157863
12.569315812724252
12.305486581196215


Mucha más distancia, realmente no cubre jugador, cubre zona

In [27]:
from shapely.geometry.point import Point
p = Point(0, 0)
circle = p.buffer(2.0)
list(circle.exterior.coords)

[(2.0, 0.0),
 (1.9903694533443939, -0.196034280659121),
 (1.961570560806461, -0.39018064403225616),
 (1.9138806714644179, -0.5805693545089242),
 (1.847759065022574, -0.7653668647301788),
 (1.7638425286967105, -0.9427934736519944),
 (1.6629392246050911, -1.1111404660392035),
 (1.5460209067254749, -1.2687865683272899),
 (1.4142135623730963, -1.4142135623730938),
 (1.2687865683272925, -1.5460209067254727),
 (1.1111404660392061, -1.6629392246050894),
 (0.9427934736519972, -1.763842528696709),
 (0.7653668647301817, -1.8477590650225726),
 (0.5805693545089272, -1.913880671464417),
 (0.39018064403225927, -1.9615705608064604),
 (0.19603428065912418, -1.9903694533443934),
 (3.2310891488651735e-15, -2.0),
 (-0.19603428065911777, -1.990369453344394),
 (-0.39018064403225294, -1.9615705608064615),
 (-0.580569354508921, -1.9138806714644188),
 (-0.7653668647301757, -1.847759065022575),
 (-0.9427934736519914, -1.763842528696712),
 (-1.111140466039201, -1.6629392246050927),
 (-1.2687865683272879, -1.546

In [28]:
point = Point(-1,-1)
circle.contains(point)

True

Con esto podemos conseguir cuantos jugadores hay alrededor

In [30]:
for frame in selected_play["frameId"].unique():
    players = selected_play[(selected_play["frameId"]==frame)&(selected_play["team"]==obtainOffenseTeam("2018090600:1037:1"))&(selected_play["route"].notnull())]
    x = unique_player[unique_player["frameId"]==frame]["x"].values[0]
    y = unique_player[unique_player["frameId"]==frame]["y"].values[0]
    p = Point(x, y)
    circle = p.buffer(5.0)
    #distances = []
    number_players = 0
    for index,row in players.iterrows():
        x_value = row["x"]
        y_value = row["y"]
        point = Point(x_value,y_value)
        if circle.contains(point):
            number_players += 1
        #line = LineString([(x, y), (x_value, y_value)]) #distancia euclidea
        #distances.append(line.length) #habría que tener en cuenta el jugador???
    print(number_players)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [32]:
for frame in selected_play["frameId"].unique():
    players = selected_play[(selected_play["frameId"]==frame)&(selected_play["team"]==obtainOffenseTeam("2018090600:1037:1"))&(selected_play["route"].notnull())]
    x = unique_player_safety[unique_player_safety["frameId"]==frame]["x"].values[0]
    y = unique_player_safety[unique_player_safety["frameId"]==frame]["y"].values[0]
    p = Point(x, y)
    circle = p.buffer(5.0)
    #distances = []
    number_players = 0
    for index,row in players.iterrows():
        x_value = row["x"]
        y_value = row["y"]
        point = Point(x_value,y_value)
        if circle.contains(point):
            number_players += 1
        #line = LineString([(x, y), (x_value, y_value)]) #distancia euclidea
        #distances.append(line.length) #habría que tener en cuenta el jugador???
    print(number_players)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [53]:
selected_play = week1[week1["id"]=="2018090600:344:1"]
selected_play

,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
4535,2018090600:344:1,15.34,37.77,0.49,1.19,0.05,96.30,27.37,None,2506467.0,Darren Sproles,43.0,RB,1,home,right,OUT
4536,2018090600:344:1,16.87,44.18,0.02,0.02,0.01,95.62,93.93,None,2507763.0,Mike Wallace,14.0,WR,1,home,right,GO
4537,2018090600:344:1,12.88,29.80,0.01,0.01,0.01,85.48,102.51,None,2532842.0,Nick Foles,9.0,QB,1,home,right,NaN
4538,2018090600:344:1,19.67,8.12,0.19,0.31,0.02,308.29,83.10,None,2539334.0,Desmond Trufant,21.0,CB,1,away,right,NaN
4539,2018090600:344:1,19.04,44.36,0.10,0.95,0.01,170.34,14.90,None,2539653.0,Robert Alford,23.0,CB,1,away,right,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,2018090600:344:1,23.81,37.30,6.17,1.99,0.61,359.86,357.77,None,2555255.0,De'Vondre Campbell,59.0,OLB,45,away,right,NaN
5206,2018090600:344:1,36.56,30.34,5.30,1.80,0.53,350.11,5.05,None,2555543.0,Keanu Neal,22.0,FS,45,away,right,NaN
5207,2018090600:344:1,23.98,21.34,4.90,0.52,0.49,349.17,0.82,None,2556445.0,Brian Poole,34.0,CB,45,away,right,NaN


In [54]:
selected_play[selected_play["frameId"]==1]

,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
4535,2018090600:344:1,15.34,37.77,0.49,1.19,0.05,96.30,27.37,None,2506467.0,Darren Sproles,43.0,RB,1,home,right,OUT
4536,2018090600:344:1,16.87,44.18,0.02,0.02,0.01,95.62,93.93,None,2507763.0,Mike Wallace,14.0,WR,1,home,right,GO
4537,2018090600:344:1,12.88,29.80,0.01,0.01,0.01,85.48,102.51,None,2532842.0,Nick Foles,9.0,QB,1,home,right,NaN
4538,2018090600:344:1,19.67,8.12,0.19,0.31,0.02,308.29,83.10,None,2539334.0,Desmond Trufant,21.0,CB,1,away,right,NaN
4539,2018090600:344:1,19.04,44.36,0.10,0.95,0.01,170.34,14.90,None,2539653.0,Robert Alford,23.0,CB,1,away,right,NaN
4540,2018090600:344:1,15.52,24.62,0.00,0.00,0.00,67.59,45.20,None,2540158.0,Zach Ertz,86.0,TE,1,home,right,IN
4541,2018090600:344:1,23.21,37.58,2.34,0.44,0.23,268.18,5.89,None,2543850.0,Ricardo Allen,37.0,SS,1,away,right,NaN
4542,2018090600:344:1,18.73,24.05,0.11,0.82,0.01,307.63,327.70,None,2552301.0,Vic Beasley,44.0,OLB,1,away,right,NaN
4543,2018090600:344:1,16.27,8.30,0.04,0.02,0.01,88.62,180.07,None,2552600.0,Nelson Agholor,13.0,WR,1,home,right,IN


In [55]:
selected_play[selected_play["event"]=="ball_snap"]["frameId"].values[0]

11

In [56]:
selected_play[selected_play["event"]=="pass_forward"]["frameId"].values[0]

34

In [57]:
selected_play.drop(selected_play[(selected_play["frameId"]<11)|(selected_play["frameId"]>34)].index,inplace=True)

/home/villamor_coto/.local/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [58]:
unique_player_lb = selected_play[selected_play["nflId"]==2555162]
unique_player_lb.head()

,id,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
index,,,,,,,,,,,,,,,,,
4694,2018090600:344:1,22.16,25.22,0.08,0.08,0.01,272.88,144.33,ball_snap,2555162.0,Deion Jones,45.0,MLB,11,away,right,NaN
4709,2018090600:344:1,22.17,25.21,0.07,0.08,0.01,272.11,144.27,None,2555162.0,Deion Jones,45.0,MLB,12,away,right,NaN
4724,2018090600:344:1,22.19,25.18,0.08,0.03,0.04,268.82,146.76,None,2555162.0,Deion Jones,45.0,MLB,13,away,right,NaN
4739,2018090600:344:1,22.19,25.15,0.14,0.85,0.02,265.07,154.10,None,2555162.0,Deion Jones,45.0,MLB,14,away,right,NaN
4754,2018090600:344:1,22.20,25.13,0.32,1.78,0.03,262.29,161.78,None,2555162.0,Deion Jones,45.0,MLB,15,away,right,NaN


In [59]:
for frame in selected_play["frameId"].unique():
    players = selected_play[(selected_play["frameId"]==frame)&(selected_play["team"]==obtainOffenseTeam("2018090600:344:1"))&(selected_play["route"].notnull())]
    x = unique_player_lb[unique_player_lb["frameId"]==frame]["x"].values[0]
    y = unique_player_lb[unique_player_lb["frameId"]==frame]["y"].values[0]
    p = Point(x, y)
    circle = p.buffer(5.0)
    #distances = []
    number_players = 0
    for index,row in players.iterrows():
        x_value = row["x"]
        y_value = row["y"]
        point = Point(x_value,y_value)
        if circle.contains(point):
            number_players += 1
        #line = LineString([(x, y), (x_value, y_value)]) #distancia euclidea
        #distances.append(line.length) #habría que tener en cuenta el jugador???
    print(number_players)

0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
2
2
2
2
1


In [60]:
for frame in selected_play["frameId"].unique():
    players = selected_play[(selected_play["frameId"]==frame)&(selected_play["team"]==obtainOffenseTeam("2018090600:344:1"))&(selected_play["route"].notnull())]
    x = unique_player_lb[unique_player_lb["frameId"]==frame]["x"].values[0]
    y = unique_player_lb[unique_player_lb["frameId"]==frame]["y"].values[0]
    p = Point(x, y)
    circle = p.buffer(5.0)
    distances = []
    number_players = 0
    for index,row in players.iterrows():
        x_value = row["x"]
        y_value = row["y"]
        point = Point(x_value,y_value)
        if circle.contains(point):
            number_players += 1
            line = LineString([(x, y), (x_value, y_value)]) #distancia euclidea
            distances.append(line.length) #habría que tener en cuenta el jugador???
    print(number_players,distances)

0 []
0 []
0 []
0 []
0 []
0 []
0 []
0 []
0 []
0 []
1 [4.727123861292402]
1 [4.310104407088069]
1 [3.898409932267256]
1 [3.5384177254812634]
1 [3.271238297648155]
1 [3.1532998588779977]
1 [3.1897648816174535]
1 [3.3562628025826564]
1 [3.63455636907725]
2 [4.837189266505911, 3.9783664989540632]
2 [4.65, 4.32801340108831]
2 [4.487694285487819, 4.675307476519594]
2 [4.308375563945186, 4.979809233293982]
1 [4.189510711288371]
